# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,motygino,58.1831,94.7592,2.10,98,100,4.47,RU,1717017270
1,1,gamay,12.3854,125.2979,24.19,95,100,1.35,PH,1717017270
2,2,toliara,-23.3500,43.6667,20.97,61,0,4.74,MG,1717017270
3,3,puerto baquerizo moreno,-0.9000,-89.6000,26.55,65,20,6.17,EC,1717017271
4,4,adamstown,-25.0660,-130.1015,21.17,70,100,3.01,PN,1717017087


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
import branca.colormap as cm
import folium
from folium import plugins

# Configure the map
colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow', 'red'], vmin=0, vmax=100)

# Configure the map plot
map_center = [0, 0]  # Centering the map at the equator
mymap = folium.Map(location=map_center, zoom_start=2)

# Add a point for each city in the DataFrame
for _, city in city_data_df.iterrows():
    # Extract latitude, longitude, and humidity
    lat = city["Lat"]
    lon = city["Lng"]
    humidity = city["Humidity"]
    
    # Determine the color based on humidity
    color = colormap(humidity)
    
    # Add a circle marker to the map
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity / 10,  # Adjust the radius scale as needed
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f'City: {city["City"]}\nHumidity: {humidity}%'
    ).add_to(mymap)

# Add the color map legend to the map
colormap.caption = 'Humidity (%)'
mymap.add_child(colormap)

# Display the map
mymap

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (city_data_df['Humidity'] > 30) & (city_data_df['Max Temp'].between(20, 40))

# Apply the criteria to filter the DataFrame using loc
filtered_city_data_df = city_data_df.loc[criteria]

# Ensure no rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
print(filtered_city_data_df.head())

# Filter the DataFrame based on criteria
filtered_city_data_df = city_data_df[criteria]

# Drop any rows with null values
filtered_city_data_df.dropna(inplace=True)

# Display sample data
filtered_city_data_df.head()

   City_ID                     City      Lat       Lng  Max Temp  Humidity  \
1        1                    gamay  12.3854  125.2979     24.19        95   
2        2                  toliara -23.3500   43.6667     20.97        61   
3        3  puerto baquerizo moreno  -0.9000  -89.6000     26.55        65   
4        4                adamstown -25.0660 -130.1015     21.17        70   
8        8                 westport  41.1415  -73.3579     25.72        52   

   Cloudiness  Wind Speed Country        Date  
1         100        1.35      PH  1717017270  
2           0        4.74      MG  1717017270  
3          20        6.17      EC  1717017271  
4         100        3.01      PN  1717017087  
8           0        4.63      US  1717017061  


/var/folders/49/dwr6y0hj2ks1d2zvzxbjk6080000gn/T/ipykernel_51994/30384809.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_city_data_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,gamay,12.3854,125.2979,24.19,95,100,1.35,PH,1717017270
2,2,toliara,-23.3500,43.6667,20.97,61,0,4.74,MG,1717017270
3,3,puerto baquerizo moreno,-0.9000,-89.6000,26.55,65,20,6.17,EC,1717017271
4,4,adamstown,-25.0660,-130.1015,21.17,70,100,3.01,PN,1717017087
8,8,westport,41.1415,-73.3579,25.72,52,0,4.63,US,1717017061


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

                      City Country      Lat       Lng  Humidity Hotel Name
1                    gamay      PH  12.3854  125.2979        95           
2                  toliara      MG -23.3500   43.6667        61           
3  puerto baquerizo moreno      EC  -0.9000  -89.6000        65           
4                adamstown      PN -25.0660 -130.1015        70           
8                 westport      US  41.1415  -73.3579        52           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000 
api_key = geoapify_key 
params = {
    'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
palmas - nearest hotel: Hotel Triângulo Mineiro


,City,Country,Lat,Lng,Humidity,Hotel Name
1,gamay,PH,12.3854,125.2979,95,No hotel found
2,toliara,MG,-23.3500,43.6667,61,Ambary
3,puerto baquerizo moreno,EC,-0.9000,-89.6000,65,Dory's House
4,adamstown,PN,-25.0660,-130.1015,70,No hotel found
8,westport,US,41.1415,-73.3579,52,Norwalk Inn and Conference Center
...,...,...,...,...,...,...
598,claveria,PH,18.6074,121.0832,85,Lakay-Lakay Beach Resort
600,rio preto da eva,BR,-2.6994,-59.6997,85,Hotel Santo Antonio
603,marine corps base hawaii - mcbh,US,21.4453,-157.7513,70,No hotel found
604,niquero,CU,20.0431,-77.5828,59,Hotel Niquero


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_center = [0, 0]
mymap = folium.Map(location=map_center, zoom_start=2)

# Add a point for each city in the DataFrame with additional information in the hover message
for _, city in hotel_df.iterrows():
    lat = city["Lat"]
    lon = city["Lng"]
    humidity = city["Humidity"]
    hotel_name = city["Hotel Name"]
    country = city["Country"]
    
    # Determine the color based on humidity
    colormap = folium.LinearColormap(['blue', 'green', 'yellow', 'red'], vmin=0, vmax=100)
    color = colormap(humidity)
    
    # Add a circle marker to the map with additional hover information
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity / 10,  # Adjust the radius scale as needed
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f'City: {city["City"]}<br>Country: {country}<br>Humidity: {humidity}%<br>Hotel: {hotel_name}'
    ).add_to(mymap)

# Display the map
mymap